# Preparación

Actualizamos la librería de Fastai a Fastai 2 y reiniciamos el entorno de ejecución

In [ ]:
!pip install -U fastai

     |████████████████████████████████| 194kB 2.8MB/s 
     |████████████████████████████████| 51kB 6.5MB/s 
  Found existing installation: fastai 1.0.61
    Uninstalling fastai-1.0.61:
      Successfully uninstalled fastai-1.0.61


In [ ]:
from fastai.vision.all import *

In [ ]:
print(fastai.__version__)

2.0.15


# Carga de datos

Definimos el path donde se escuentra nuestro dataset y creamos el dataLoader. En el dataloader utilizamos el [data augmentation](https://docs.fast.ai/vision.augment#aug_transforms) por defecto de fastai.

El dataset tiene que estar organizado de la siguiente forma:

/path

> /train

>>    /0

>>    /1

 > /valid

 >>   /0
 
 >>   /1

In [ ]:
!wget https://unirioja-my.sharepoint.com/:u:/g/personal/adines_unirioja_es/EfYzuKTZEPFLo3DmDqAZM3QB8aMHvn2HKrq2ch4OK5Nh3A?download=1 -O melanoma.zip
!unzip melanoma.zip

In [ ]:
path="melanoma"

In [ ]:
bs=32

In [ ]:
size=224

In [ ]:
data=ImageDataLoaders.from_folder(path,batch_tfms=aug_transforms(),item_tfms=Resize(size),bs=bs)

Al estar el dataset tan poco balanceado (32500 imágenes con clase 0 y 500 con clase 1) podemos utilizar una función de pérdida con pesos. Para ello creamos una función que calcule los pesos adecuados de las clases.

In [ ]:
def get_weights(dls):
    classes = [0,1]]
    train_lbls = L(map(lambda x: classes[x[1]], dls.train_ds))
    label_counter = Counter(train_lbls)
    n_most_common_class = max(label_counter.values()); 
    print(f'Occurrences of the most common class {n_most_common_class}')
    
    weights = [n_most_common_class/v for k, v in label_counter.items() if v > 0]
    return weights 

In [ ]:
weights = get_weights(data) 
class_weights = torch.FloatTensor(weights).to(data.device)

Creamos el learner con la arquitetura resnet50 y le asignamos como función de pérdida una weigthed cross entropy loss.

In [ ]:
learn=cnn_learner(data,models.resnet50,metrics='accuracy')
learn.loss_func = partial(F.cross_entropy, weight=class_weights)

Utilizamos la función fine-tune para entrenar el modelo. Esta función entrena la cabeza con el resto del modelo congelado, durante una etapa (este valor se puede cambiar freeze_epochs)

In [ ]:
learn.fine_tune(10)